<a href="https://colab.research.google.com/github/MINAADELMARKOS/SuperHerosAnalysis/blob/main/Superheors_Analysis_CreateWindowFunctions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Install Pyspark**

In [1]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=1372726befec344fa8abd8db349ca0cd1f5985df04073ca96085f26b205e35c5
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


# **Import Libraries**

In [2]:
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession
from pyspark.sql.types import Row
from pyspark.sql.functions import desc
from pyspark.sql.functions import avg

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
spark = SparkSession \
        .builder  \
        .appName("Window functions") \
        .getOrCreate()

In [9]:
sqlContext = SQLContext(spark)

In [12]:
characters_info = "/content/drive/MyDrive/Python_Learn /superheroes_info.csv"
characters_stats = "/content/drive/MyDrive/Python_Learn /superheroes_stats.csv"

In [13]:
powers = spark.read\
              .format("csv")\
              .option("header","true")\
              .load(characters_stats)

In [14]:
powers.show(4)

+----------+---------+------------+--------+-----+----------+-----+------+-----+
|      Name|Alignment|Intelligence|Strength|Speed|Durability|Power|Combat|Total|
+----------+---------+------------+--------+-----+----------+-----+------+-----+
|   3-D Man|     good|        50.0|    31.0| 43.0|      32.0| 25.0|  52.0|233.0|
|    A-Bomb|     good|        38.0|   100.0| 17.0|      80.0| 17.0|  64.0|316.0|
|Abe Sapien|     good|        88.0|    14.0| 35.0|      42.0| 35.0|  85.0|299.0|
|  Abin Sur|     good|        50.0|    90.0| 53.0|      64.0| 84.0|  65.0|406.0|
+----------+---------+------------+--------+-----+----------+-----+------+-----+
only showing top 4 rows



In [15]:
powers.columns

['Name',
 'Alignment',
 'Intelligence',
 'Strength',
 'Speed',
 'Durability',
 'Power',
 'Combat',
 'Total']

In [16]:
powers = powers.filter(powers.Alignment != 'NA')

In [17]:
powers.show(4)

+----------+---------+------------+--------+-----+----------+-----+------+-----+
|      Name|Alignment|Intelligence|Strength|Speed|Durability|Power|Combat|Total|
+----------+---------+------------+--------+-----+----------+-----+------+-----+
|   3-D Man|     good|        50.0|    31.0| 43.0|      32.0| 25.0|  52.0|233.0|
|    A-Bomb|     good|        38.0|   100.0| 17.0|      80.0| 17.0|  64.0|316.0|
|Abe Sapien|     good|        88.0|    14.0| 35.0|      42.0| 35.0|  85.0|299.0|
|  Abin Sur|     good|        50.0|    90.0| 53.0|      64.0| 84.0|  65.0|406.0|
+----------+---------+------------+--------+-----+----------+-----+------+-----+
only showing top 4 rows



In [18]:
from pyspark.sql.types import IntegerType

In [19]:
powers.show(1)

+-------+---------+------------+--------+-----+----------+-----+------+-----+
|   Name|Alignment|Intelligence|Strength|Speed|Durability|Power|Combat|Total|
+-------+---------+------------+--------+-----+----------+-----+------+-----+
|3-D Man|     good|        50.0|    31.0| 43.0|      32.0| 25.0|  52.0|233.0|
+-------+---------+------------+--------+-----+----------+-----+------+-----+
only showing top 1 row



In [20]:
import sys
from pyspark.sql.window import Window
import pyspark.sql.functions as func

In [21]:
windowSpecl = Window \
             .partitionBy(powers['Alignment']) \
             .orderBy(powers['Intelligence'].desc())

In [22]:
followers_rank = (func.rank().over(windowSpecl))

In [23]:
followers_rank = powers.select(powers['Name'],
                               powers['Intelligence'],
                               powers['Alignment']
                               ).withColumn('rank',
                                            func.rank().over(windowSpecl)

                               )

followers_rank.show(5)

+--------------+------------+---------+----+
|          Name|Intelligence|Alignment|rank|
+--------------+------------+---------+----+
|   General Zod|        94.0|      bad|   1|
|Superboy-Prime|        94.0|      bad|   1|
|Solomon Grundy|         9.0|      bad|   3|
|       Abraxas|        88.0|      bad|   4|
|  Anti-Monitor|        88.0|      bad|   4|
+--------------+------------+---------+----+
only showing top 5 rows



In [24]:
good_heros = followers_rank.filter(followers_rank['Alignment'] == 'good')
good_heros.show()

+-----------------+------------+---------+----+
|             Name|Intelligence|Alignment|rank|
+-----------------+------------+---------+----+
|        Supergirl|        94.0|     good|   1|
|             Dash|         9.0|     good|   2|
|           Krypto|         9.0|     good|   2|
|       Abe Sapien|        88.0|     good|   4|
|        Batman II|        88.0|     good|   4|
|            Beast|        88.0|     good|   4|
|         Beyonder|        88.0|     good|   4|
|    Black Panther|        88.0|     good|   4|
|            Cable|        88.0|     good|   4|
|          Century|        88.0|     good|   4|
|     Dr Manhattan|        88.0|     good|   4|
|             Hulk|        88.0|     good|   4|
|        JJ Powell|        88.0|     good|   4|
|           Metron|        88.0|     good|   4|
|        Nightwing|        88.0|     good|   4|
|       Power Girl|        88.0|     good|   4|
|          Robin I|        88.0|     good|   4|
|Scarlet Spider II|        88.0|     goo